In [2]:
import pandas as pd

In [3]:
employees_df = pd.read_csv("data/cbp13co.txt", dtype={'fipstate': object, 'fipscty': object})
codes_df = pd.read_csv("data/georef12.txt", dtype={'fipstate': object, 'fipscty': object})
land_area_df = pd.read_csv('data/DEC_00_SF1_GCTPH1.US05PR.csv', dtype={'GCT_STUB.target-geo-id2': object})

In [106]:
print employees_df

        fipstate fipscty   naics empflag emp_nf    emp qp1_nf    qp1 ap_nf  \
0             01     001  ------     NaN      G  10120      G  71267     G   
1             01     001  11----     NaN      G     33      G    277     G   
2             01     001  113///     NaN      G     33      G    277     G   
3             01     001  1133//     NaN      G     33      G    277     G   
4             01     001  11331/     NaN      G     33      G    277     G   
5             01     001  113310     NaN      G     33      G    277     G   
6             01     001  21----       B      D      0      D      0     D   
7             01     001  212///       B      D      0      D      0     D   
8             01     001  2123//       B      D      0      D      0     D   
9             01     001  21231/       B      D      0      D      0     D   
10            01     001  212319       B      D      0      D      0     D   
11            01     001  21232/       A      D      0      D   

In [11]:
# want the total over all naics codes (where naics column = '------')
# excluding statewide totals (where county code = 999)
emp_all_df = employees_df[(employees_df['naics'] == '------') & (employees_df['fipscty'] != '999')].copy()
emp_all_df.shape

(3142, 26)

In [9]:
# also want the name of the county and state
print codes_df[:5]
with_names_df = pd.merge(emp_all_df, codes_df, on=['fipstate', 'fipscty'], how='inner')

  fipstate fipscty             ctyname
0       01     001  Autauga County, AL
1       01     003  Baldwin County, AL
2       01     005  Barbour County, AL
3       01     007     Bibb County, AL
4       01     009   Blount County, AL


In [70]:
with_names_df[:5]

,fipstate,fipscty,naics,empflag,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,...,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty,ctyname
0,01,001,------,NaN,G,10120,G,71267,G,291673,...,1,1,0,0,0,0,0,63,1,"Autauga County, AL"
1,01,003,------,NaN,G,54988,G,385826,G,1668968,...,10,5,1,1,0,0,0,63,3,"Baldwin County, AL"
2,01,005,------,NaN,G,6611,H,53116,H,212757,...,3,0,1,1,0,0,0,63,5,"Barbour County, AL"
3,01,007,------,NaN,G,3145,G,27008,H,114742,...,0,1,0,0,0,0,0,63,7,"Bibb County, AL"
4,01,009,------,NaN,G,6798,G,49518,G,197060,...,1,1,0,0,0,0,0,63,9,"Blount County, AL"


In [71]:
# combine fip codes to be 5 digit code for topojson compatibility
with_names_df['id'] = with_names_df['fipstate'] + with_names_df['fipscty']

In [72]:
# only need total number of businesses with 50-99 employees
sb_df = with_names_df[['id', 'ctyname', 'n50_99']].copy()

In [73]:
sb_df[:5]

,id,ctyname,n50_99
0,01001,"Autauga County, AL",34
1,01003,"Baldwin County, AL",115
2,01005,"Barbour County, AL",8
3,01007,"Bibb County, AL",4
4,01009,"Blount County, AL",12


In [75]:
# calculate density using county land area

In [96]:
land_area_df[:5]

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HC01,HC02,HC04,HC05,HC06,HC08,HC09
0,0100000US,NaN,United States,0100000US,NaN,United States,United States,281421906,115904641,3794083.06,256644.62,3537438.44,79.6,32.8
1,0100000US,NaN,United States,0400000US01,01,United States - Alabama,Alabama,4447100,1963711,52419.02,1675.01,50744.00,87.6,38.7
2,0100000US,NaN,United States,0500000US01001,01001,United States - Alabama - Autauga County,Autauga County,43671,17662,604.45,8.48,595.97,73.3,29.6
3,0100000US,NaN,United States,0500000US01003,01003,United States - Alabama - Baldwin County,Baldwin County,140415,74285,2026.93,430.58,1596.35,88.0,46.5
4,0100000US,NaN,United States,0500000US01005,01005,United States - Alabama - Barbour County,Barbour County,29038,12461,904.52,19.61,884.90,32.8,14.1


In [97]:
# total land area is column HC04
area_df = land_area_df.rename(columns={'GCT_STUB.target-geo-id2':'id', 'HC06':'area'})[['id', 'area']]

In [98]:
# left join on fips id, convert to int
sb_area_df = pd.merge(sb_df, area_df, on=['id'], how='left')
sb_area_df['id'] = sb_area_df['id'].apply(int)

In [99]:
# calculate density
sb_area_df['density'] = sb_area_df['n50_99'] / sb_area_df['area']

In [100]:
sb_density_df = sb_area_df[['id', 'ctyname', 'density', 'n50_99']]
sb_density_df[:5]

,id,ctyname,density,n50_99
0,1001,"Autauga County, AL",0.057050,34
1,1003,"Baldwin County, AL",0.072039,115
2,1005,"Barbour County, AL",0.009041,8
3,1007,"Bibb County, AL",0.006420,4
4,1009,"Blount County, AL",0.018588,12


In [101]:
sb_density_df.to_csv('data/sbdata.csv', index=False)

In [103]:
sb_density_df['density'].describe()

count    3136.000000
mean        0.226251
std         2.776954
min         0.000000
25%         0.005515
50%         0.017610
75%         0.062110
max       147.996516
Name: density, dtype: float64